<a href="https://colab.research.google.com/github/muyuuuu/colab/blob/main/adversarial-denoise/adp(ICML%202021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.49, 0.48, 0.44), (0.24, 0.24, 0.26))])

batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class Score(nn.Module):
    def __init__(self):
        super().__init__()
        nef = 32
        self.u_net = nn.Sequential(
            # input is (nc) x 28 x 28
            nn.Conv2d(3, nef, 4, stride=2, padding=1),
            # nn.Softplus(),
            nn.GroupNorm(4, nef),
            nn.ELU(),
            # state size. (nef) x 14 x 14
            nn.Conv2d(nef, nef * 2, 4, stride=2, padding=1),
            nn.GroupNorm(4, nef * 2),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef*2) x 7 x 7
            nn.Conv2d(nef * 2, nef * 4, 5, stride=1, padding=0),
            nn.GroupNorm(4, nef * 4),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef*4) x 3 x 3
            nn.ConvTranspose2d(nef * 4, nef * 2, 5, stride=1, padding=0),
            nn.GroupNorm(4, nef * 2),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef*2) x 7 x 7
            nn.ConvTranspose2d(nef * 2, nef, 4, stride=2, padding=1),
            nn.GroupNorm(4, nef),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef) x 14 x 14
            nn.ConvTranspose2d(nef, 3, 4, stride=2, padding=1),
            # nn.Softplus()
            nn.ELU()
        )
        self.fc = nn.Sequential(
            nn.Linear(3 * 32 * 32, 1024),
            nn.LayerNorm(1024),
            nn.ELU(),
            nn.Linear(1024, 3 * 32 * 32)
        )

    def forward(self, x):

        score = self.u_net(x)
        score = self.fc(score.view(x.shape[0], -1)).view(
            x.shape[0], 3, 32, 32)
        return score

In [5]:
net = Score().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.004, momentum=0.9)
criterion = nn.MSELoss()

In [21]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, _ = data
        inputs = inputs.to(device)
        outputs = net(inputs)

        # get the inputs; data is a list of [inputs, labels]
        inputs, _ = data
        a = torch.normal(0, 0.1, inputs.size())
        inputs, a = inputs.to(device), a.to(device) 
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs + a)
        # print(torch.max(outputs), torch.min(outputs))
        loss = criterion(outputs, a)

        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 500 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

print('Finished Training')

[1,   500] loss: 0.010
[1,  1000] loss: 0.010
[1,  1500] loss: 0.010
[2,   500] loss: 0.010


KeyboardInterrupt: ignored

In [ ]:
torch.save(net, 'score.pth')

In [ ]:
delta = 1e-5
lambda_ = 0.05
tao = 0.001
alpha = None
correct = 0
total = 0


with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        predicted = torch.zeros((10, 4))
        for i in range (10):
            a = torch.normal(0, 0.1, (batch_size, 3, 32, 32)).to(device)
            inputs = inputs.to(device)
            inputs += a
            for _ in range (10):
                # print(net(inputs))                
                inputs_ = inputs + delta * net(inputs)
                alpha = lambda_ * delta / (1 - ( net(inputs) * net(inputs_) / torch.square(net(inputs) )))
                inputs = inputs + alpha * net(inputs)
                if (torch.norm(net(inputs), p=1) < tao):
                    break
            break
        break
    #         # 预测 inputs
    #         outputs = model(inputs)
    #         _, idx = torch.max(outputs.data, 1)
    #         predicted[i] = idx

    #     # predicted, _ = torch.max(predicted, 0)
    #     predicted, _ = torch.mode(predicted, 0)
    #     # break
    #     total += labels.size(0)
    #     correct += (predicted == labels).sum().item()

    # print('Accuracy of the network on the 10000 test images: %d %%' % (
    #     100 * correct / total))

In [ ]:
a = torch.randint(10, (10, 4,))
a

In [ ]:
b = torch.mode(a, dim=0)

In [ ]:
b